In [1]:
#Init enviroment
!python3 -m venv OXTC
!source OXTC/bin/activate
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Generate the onnx model
%run generate_onnx_model.py

Creating model with in_size=5 and out_size=3
Model successfully created.
Model saved to ./simple_model_5_3.onnx

Running the model to get test data
Loading model from ./simple_model_5_3.onnx
Input x: shape=[1, 5]
Output y: shape=[1, 3]
Output y_hat: shape=[1, 3]

Weights:
[[10 -6  9]
 [-6 -3 -4]
 [ 0 19  8]
 [-1  1 17]
 [ 8  8 -1]]

Input:
x:	 [[0 0 0 0 0]]

Outputs:
y:	 [[0 0 0]]
y_hat:	 [[0 0 0]]


In [8]:
#Load model
import onnx
import numpy as np
import tvm
from tvm import te
import tvm.relay as relay
import tvm.micro
import tvm.contrib.utils
from tvm.micro import export_model_library_format

onnx_model = onnx.load("simple_model_5_3.onnx")

ImportError: micro tvm is not enabled. Set USE_MICRO to ON in config.cmake

In [7]:
RUNTIME = tvm.relay.backend.Runtime("crt", {"system-lib": True})
TARGET = tvm.micro.testing.get_target("crt")

input_name = "x"
shape_dict = {"x": (5,)}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

with tvm.transform.PassContext(opt_level=3, config={"tir.disable_vectorize": True}):
    module = relay.build(mod, target=TARGET, runtime=RUNTIME, params=params)

AttributeError: module 'tvm' has no attribute 'micro'